<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/coxph_find_best_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [ ]:
pip install pycox

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [ ]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [ ]:
dataset_val = dataset.sample(frac=0.2)
dataset_train = dataset.drop(dataset_val.index)
dataset_test = dataset_train.sample(frac=0.2)
dataset_train = dataset_train.drop(dataset_test.index)

- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [ ]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [ ]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [ ]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [ ]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [ ]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [1,2,3]
number_nodes = [32, 64, 128, 256, 512]
learning_rates = [0.0001, 0.001, 0.01, 0.1]
brier_scores = []

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)

      batch_size = 256
      model.optimizer.set_lr(k)
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)
      
      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)
      brier_scores.append([score, i, j, k, treat_hr[:10]])

1
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
0:	[0s / 0s],		train_loss: 4.4012,	val_loss: 4.3335
1:	[0s / 0s],		train_loss: 4.3771,	val_loss: 4.3354
2:	[0s / 0s],		train_loss: 4.3710,	val_loss: 4.3377
3:	[0s / 0s],		train_loss: 4.3428,	val_loss: 4.3405
4:	[0s / 0s],		train_loss: 4.3449,	val_loss: 4.3439
5:	[0s / 0s],		train_loss: 4.3571,	val_loss: 4.3480
6:	[0s / 0s],		train_loss: 4.3351,	val_loss: 4.3530
7:	[0s / 0s],		train_loss: 4.3039,	val_loss: 4.3589
8:	[0s / 0s],		train_loss: 4.3026,	val_loss: 4.3660
9:	[0s / 0s],		train_loss: 4.2585,	val_loss: 4.3743
10:	[0s / 0s],		train_loss: 4.2567,	val_loss: 4.3838


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


1
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.58 µs
0:	[0s / 0s],		train_loss: 4.4702,	val_loss: 4.3198
1:	[0s / 0s],		train_loss: 4.2998,	val_loss: 4.3158
2:	[0s / 0s],		train_loss: 4.1681,	val_loss: 4.3114
3:	[0s / 0s],		train_loss: 4.0610,	val_loss: 4.3065
4:	[0s / 0s],		train_loss: 4.0043,	val_loss: 4.3008
5:	[0s / 0s],		train_loss: 3.9647,	val_loss: 4.2951
6:	[0s / 0s],		train_loss: 3.8915,	val_loss: 4.2893
7:	[0s / 0s],		train_loss: 3.8258,	val_loss: 4.2840
8:	[0s / 0s],		train_loss: 3.7839,	val_loss: 4.2788
9:	[0s / 1s],		train_loss: 3.7365,	val_loss: 4.2744
10:	[0s / 1s],		train_loss: 3.6721,	val_loss: 4.2712
11:	[0s / 1s],		train_loss: 3.6338,	val_loss: 4.2706
12:	[0s / 1s],		train_loss: 3.6045,	val_loss: 4.2734
13:	[0s / 1s],		train_loss: 3.5270,	val_loss: 4.2791
14:	[0s / 1s],		train_loss: 3.5369,	val_loss: 4.2870
15:	[0s / 1s],		train_loss: 3.4609,	val_loss: 4.2965
16:	[0s / 1s],		train_loss: 3.4164,	val_loss: 4.3094
17:	[0s / 1s],		train_loss: 3.3811,	val_los

- brier_score가 가장 작은 것부터 정렬


In [ ]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][4][i][1])

- brier_score가 가장 좋은 케이스 출력

In [ ]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.08456742288041566
Hidden Layers : 2
Number of Nodes : 64
Learning Rate : 0.01
Selection : ['G181', 'G79', 'G86', 'G280', 'G244', 'G187', 'G107', 'G248', 'G83', 'G214']
